In [1]:
import pandas
import numpy as np
import scipy
import time
import datetime

In [54]:
data = pandas.read_csv('./data/final_features.csv', index_col='match_id')

## Подход 1: градиентный бустинг "в лоб"

Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [3]:
X = data.drop([
    'radiant_win', 'duration', 'tower_status_radiant', 
    'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'
], axis = 1)

#### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [4]:
counts = X.count()
print(counts[counts != X.shape[0]])

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


Пропуски имеют признаки, отмечающие первую покупку опциональных игровых предметов, например курьера radiant_courier_time, а также отмечающие первую смерть first_blood_time. Причина - эти события просто не успели произойти.

#### Как называется столбец, содержащий целевую переменную?

In [5]:
y = data.loc[:, 'radiant_win']

radiant_win

#### Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?  Какое качество при этом получилось?

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve

In [7]:
scores = []
validator = KFold(y.size, n_folds=5, shuffle=True, random_state=1)
X_filled = X.fillna(-1)
for n_est in [5, 10, 20, 30, 40]:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est, random_state=1)
    scores.append((
        n_est,
        cross_val_score(
             clf, X_filled, y=y, cv=validator, scoring=lambda est, X, y: roc_auc_score(y, est.predict_proba(X)[:, 1])
        ).mean(),
        str(datetime.datetime.now() - start_time)
    ))

In [8]:
scores_df = pandas.DataFrame.from_records(scores)
scores_df.columns = ['n_est', 'cross_val_score', 'time']
print(scores_df)

   n_est  cross_val_score            time
0      5         0.635672  0:00:27.564635
1     10         0.664833  0:00:44.263637
2     20         0.682114  0:01:24.450218
3     30         0.689695  0:01:58.060409
4     40         0.693934  0:02:36.256619


Кросс-валидация с 30 деревьями в моем случае длилась 1,5 минуты. Качество при этом - 69%, что непохоже на переобучение.

#### Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

В данном случае повышать количество деревьев особого смысла не имеет - от 30 до 40 качество растет всего на 0.5%, а дальше наверняка начнет падать. Обучение - заметно дольше.

Для ускорения обечения подходят методы из инструкции - брать не всю выборку, а некоторое ее случайное подмножество;  уменьшить глубину деревьев.

## Подход 2: логистическая регрессия

#### Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

In [18]:
scores = []
scaler = StandardScaler()
X_filled_scaled = scaler.fit_transform(X.fillna(0))
validator = KFold(y.size, n_folds=5, shuffle=True, random_state=1)

for C in [0.0001, 0.001, 0.01, 0.1, 0.1, 1., 10., 100.]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=C, penalty='l2', random_state=1)
    scores.append((
        C,
        cross_val_score(
             clf, X_filled_scaled, y=y, cv=validator, scoring=lambda est, X, y: roc_auc_score(y, est.predict_proba(X)[:, 1])
        ).mean(),
        str(datetime.datetime.now() - start_time)
    ))

In [19]:
scores_df = pandas.DataFrame.from_records(scores)
scores_df.columns = ['C', 'cross_val_score', 'time']
print(scores_df)

          C  cross_val_score            time
0    0.0001         0.711287  0:00:06.012619
1    0.0010         0.716210  0:00:10.952714
2    0.0100         0.716376  0:00:15.320015
3    0.1000         0.716351  0:00:15.410833
4    0.1000         0.716351  0:00:15.573950
5    1.0000         0.716347  0:00:15.962781
6   10.0000         0.716347  0:00:15.511846
7  100.0000         0.716347  0:00:15.418847


In [17]:
kf = KFold(y.size, n_folds=5, shuffle=True, random_state=1)
clf = LogisticRegression(penalty='l2', random_state=1)
grid={'C': np.power(10.0, np.arange(-5.0, 6.0))}
gs = GridSearchCV(clf, grid, scoring = 'roc_auc', cv = kf)
start_time = datetime.datetime.now()
gs.fit(StandardScaler().fit_transform(X.fillna(0)), y)
scores = []
for scrs in gs.grid_scores_:
    scores.append((scrs.parameters['C'], scrs.mean_validation_score))
print 'Time elapsed:', datetime.datetime.now() - start_time
print scores
clf = gs.best_estimator_

Time elapsed: 0:02:35.683153
[(1.0000000000000001e-05, 0.69517709826221419), (0.0001, 0.71128651271563847), (0.001, 0.71620963600990128), (0.01, 0.71637579485354486), (0.10000000000000001, 0.71635092675197554), (1.0, 0.7163472681900791), (10.0, 0.71634707349810633), (100.0, 0.71634707670546227), (1000.0, 0.71634708942298131), (10000.0, 0.71634710107483035), (100000.0, 0.71634710213378894)]


Логистическая регрессия работает быстрее, но вероятно где-то допустил ошибку, и качество всегда одинаковое - 71.63%, независимо от параметра C. Качество чуть лучше, чем при градиентном бустинге.

#### Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем можно объяснить это изменение?

In [21]:
X_ = X.drop([
    'lobby_type',
    'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
    'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'
], axis = 1)
scores = []
scaler = StandardScaler()
X_filled_scaled = scaler.fit_transform(X_.fillna(0))
validator = KFold(y.size, n_folds=5, shuffle=True, random_state=1)
for C in [0.01, 0.1, 0.1, 1., 10., 100.]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=C, penalty='l2', random_state=1)
    scores.append((
        C,
        cross_val_score(
             clf, X_filled_scaled, y=y, cv=validator, scoring=lambda est, X, y: roc_auc_score(y, est.predict_proba(X)[:, 1])
        ).mean(),
        str(datetime.datetime.now() - start_time)
    ))

In [22]:
scores_df = pandas.DataFrame.from_records(scores)
scores_df.columns = ['C', 'cross_val_score', 'time']
print(scores_df)

        C  cross_val_score            time
0    0.01         0.716409  0:00:12.782082
1    0.10         0.716384  0:00:13.416932
2    0.10         0.716384  0:00:13.189054
3    1.00         0.716380  0:00:13.161176
4   10.00         0.716380  0:00:15.110385
5  100.00         0.716380  0:00:13.754142


Где-то та же ненаходимая ошибка, потом качество почти то-же: 71,64%, т.е. на 0.01% лучше.

#### Сколько различных идентификаторов героев существует в данной игре?

In [36]:
N = max(data[[
    'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
    'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'
]].values.ravel())
print(N)

112


#### Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем можно это объяснить?

In [55]:
X_pick = np.zeros((data.shape[0], N))
for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1     
X_nocat = StandardScaler().fit_transform(X.drop([
    'lobby_type',
    'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
    'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'
], axis = 1).fillna(0))
X_filled_scaled = pandas.concat([pandas.DataFrame(X_nocat), pandas.DataFrame(X_pick)], axis=1)

In [51]:
scores = []
validator = KFold(y.size, n_folds=5, shuffle=True, random_state=1)
for C in [0.01, 0.1, 0.1, 1., 10., 100.]:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=C, penalty='l2', random_state=1)
    scores.append((
        C,
        cross_val_score(
             clf, X_filled_scaled, y=y, cv=validator, scoring=lambda est, X, y: roc_auc_score(y, est.predict_proba(X)[:, 1])
        ).mean(),
        str(datetime.datetime.now() - start_time)
    ))

In [52]:
scores_df = pandas.DataFrame.from_records(scores)
scores_df.columns = ['C', 'cross_val_score', 'time']
print(scores_df)

        C  cross_val_score            time
0    0.01         0.751652  0:00:23.335311
1    0.10         0.751873  0:00:30.554342
2    0.10         0.751873  0:00:28.292895
3    1.00         0.751857  0:00:29.215879
4   10.00         0.751856  0:00:32.147987
5  100.00         0.751856  0:00:30.703166


Опять никакой разницы, и склеить датафреймы не получилось.

#### Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [56]:
clf = LogisticRegression(C=.1, penalty='l2', random_state=1)
scaler = StandardScaler()
X_filled_scaled = scaler.fit_transform(X.fillna(0))
clf.fit(X_filled_scaled, y)
predictions = clf.predict_proba(scaler.transform(X_test))
print(min(predictions))
print(max(predictions))

NameError: name 'X_test' is not defined

In [ ]:
print(max(predictions[:, 0]))
print(min(predictions[:, 0]))